In [374]:
import numpy as np
from PIL import Image
import random


In [375]:
def int_of_bytes(bytes):
    return int.from_bytes(bytes, "big")


def open_image(path):
    f = open(path, "rb")
    f.read(4)  # magic number
    num_images = int_of_bytes(f.read(4))
    rows = int_of_bytes(f.read(4))
    columns = int_of_bytes(f.read(4))
    f.close()
    data = np.fromfile(path, dtype="uint8")
    data = data[16:]
    return np.reshape(data, (num_images, rows, columns))


def display_image(np_array):
    im = Image.fromarray(np_array)
    im = im.convert("L")
    display(im)


def open_label(path):
    f = open(path, "rb")
    f.read(4)  # magic number
    num_images = int_of_bytes(f.read(4))
    f.close()
    data = np.fromfile(path, dtype="uint8")
    data = data[8:]
    return np.reshape(data, (num_images))


In [376]:
images_train = open_image("train-images-idx3-ubyte")
labels_train = open_label("train-labels-idx1-ubyte")
images_test = open_image("t10k-images-idx3-ubyte")
labels_test = open_label("t10k-labels-idx1-ubyte")


In [377]:
def p_sigmoid(x):
    y = sigmoid(x)
    return y * (1 - y)


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def prepare_data(images, labels):
    return [(i.reshape(i.size, 1), label_to_vector(l)) for i, l in zip(images, labels)]


def label_to_vector(label: int):
    r = np.zeros((10, 1))
    r[label][0] = 1
    return r


def vector_to_label(vec):
    return np.argmax(vec)


In [378]:
class Network:
    def __init__(self, sizes):
        self.sizes = sizes
        self.num_layers = len(sizes)
        self.biases = [np.random.randn(x, 1) for x in sizes[1:]]
        self.weights = [np.random.randn(x, y) for x, y in zip(sizes[1:], sizes)]

    def sdg(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        n = len(training_data)
        for i in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[x : x + mini_batch_size]
                for x in range(0, n, mini_batch_size)
            ]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print(f"Epoch {i}: {self.evaluate(test_data)} / {len(test_data)}")
            else:
                print(f"Epoch {i} complete")

    def update_mini_batch(self, mini_batch, eta):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.biases = [
            b - (eta / len(mini_batch)) * nb for b, nb in zip(self.biases, nabla_b)
        ]
        self.weights = [
            w - (eta / len(mini_batch)) * nw for w, nw in zip(self.weights, nabla_w)
        ]

    def backprop(self, x, y):
        zs = []
        activations = [x]
        activation = x
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for w, b in zip(self.weights, self.biases):
            z = np.dot(w, activation) + b
            activation = sigmoid(z)
            zs.append(z)
            activations.append(activation)
        delta = (activations[-1] - y) * p_sigmoid(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = delta @ activations[-2].transpose()

        for i in range(2, self.num_layers):
            delta = (
                self.weights[-i + 1].transpose() @ delta
            ) * p_sigmoid(zs[-i])
            nabla_b[-i] = delta
            nabla_w[-i] = delta @ activations[-i - 1].transpose()
        return nabla_b, nabla_w

    def evaluate(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == vector_to_label(y)) for (x, y) in test_results)

    def feedforward(self, inp):
        for b, w in zip(self.biases, self.weights):
            inp = sigmoid(np.dot(w, inp) + b)
        return inp


In [379]:
network = Network([784, 200, 80, 10])


def main():
    training_data = prepare_data(images_train, labels_train)
    test_data = prepare_data(images_test, labels_test)
    network.sdg(training_data, 30, 20, 0.5, test_data=test_data)


In [380]:
main()


/var/folders/k8/c8_zkt093fv9l1wmmxjpjt100000gn/T/ipykernel_48999/4275333500.py:7: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch 0: 4738 / 10000
Epoch 1: 6836 / 10000
Epoch 2: 7513 / 10000
Epoch 3: 7722 / 10000
Epoch 4: 7920 / 10000
Epoch 5: 7859 / 10000
Epoch 6: 7598 / 10000
Epoch 7: 7972 / 10000
Epoch 8: 7933 / 10000
Epoch 9: 7807 / 10000
Epoch 10: 7812 / 10000
Epoch 11: 7817 / 10000
Epoch 12: 8145 / 10000
Epoch 13: 8141 / 10000
Epoch 14: 8011 / 10000
Epoch 15: 8000 / 10000
Epoch 16: 8054 / 10000
Epoch 17: 8098 / 10000
Epoch 18: 8303 / 10000
Epoch 19: 8106 / 10000
Epoch 20: 8269 / 10000
Epoch 21: 8190 / 10000
Epoch 22: 8284 / 10000
Epoch 23: 7982 / 10000
Epoch 24: 8091 / 10000
Epoch 25: 8215 / 10000
Epoch 26: 8344 / 10000
Epoch 27: 8123 / 10000
Epoch 28: 8197 / 10000
Epoch 29: 8093 / 10000


In [381]:
def predict_label(image):
    vl = network.feedforward(image.reshape(image.size, 1))
    return vector_to_label(vl)


In [383]:
display_image(images_test[1022])

In [384]:
predict_label(images_test[1022])

/var/folders/k8/c8_zkt093fv9l1wmmxjpjt100000gn/T/ipykernel_48999/4275333500.py:7: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


5

In [385]:
network.evaluate(test_data=prepare_data(images_test, labels_test))

/var/folders/k8/c8_zkt093fv9l1wmmxjpjt100000gn/T/ipykernel_48999/4275333500.py:7: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


8093